In [1]:
import io
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from tqdm import tqdm
import random

In [2]:
data = pd.read_csv('E:/DS5500_Project/features_v2.csv')
data

,session,trial,filename,crest_factor_F1,dasd_F1,diffvar_F1,form_factor_F1,iemg_F1,kurtosis_F1,kurtosis_f_F1,...,rms_W9,skew_W9,skew_f_W9,ssi_W9,sum_f_W9,var_f_W9,wflen_W9,willison_W9,gesture,participant
0,1,1,E:\DS5500-project\data\gesture-recognition-and...,7.091099,0.021834,0.000606,-6502.754507,365.456658,2.574033,85.547962,...,0.022588,-0.148877,5.529415,5.224663,0.109574,2.901826e-10,69.944068,15.0,10,1
1,1,2,E:\DS5500-project\data\gesture-recognition-and...,13.324514,0.023852,0.000689,1352.897349,379.664639,16.942218,329.114870,...,0.016172,-0.205427,9.332349,2.677940,0.077566,2.769772e-10,54.530919,23.0,10,1
2,1,3,E:\DS5500-project\data\gesture-recognition-and...,8.047954,0.016213,0.000326,1295.052394,296.823135,3.795827,134.751765,...,0.017039,-0.204604,9.078430,2.973002,0.064458,1.426359e-10,57.815909,13.0,10,1
3,1,4,E:\DS5500-project\data\gesture-recognition-and...,8.459555,0.026572,0.000864,14453.068135,479.688214,6.938354,98.832790,...,0.031982,-0.266818,5.067522,10.473811,0.131909,4.509337e-10,83.441138,0.0,10,1
4,1,5,E:\DS5500-project\data\gesture-recognition-and...,8.329011,0.026896,0.000901,11827.310277,464.350257,2.379207,116.412205,...,0.031134,-0.092205,5.414199,9.925787,0.177516,9.577100e-10,94.393506,3.0,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15346,3,3,E:\DS5500-project\data\gesture-recognition-and...,7.771453,0.013208,0.000208,2599.892725,274.024844,2.176596,28.888640,...,0.020526,-0.293608,3.830298,4.314177,0.035283,1.966555e-11,40.970398,2.0,9,9
15347,3,4,E:\DS5500-project\data\gesture-recognition-and...,9.172642,0.019135,0.000429,1255.242161,385.695263,4.615531,133.237150,...,0.025825,-0.389927,4.200155,6.829604,0.076162,1.003953e-10,52.342457,0.0,9,9
15348,3,5,E:\DS5500-project\data\gesture-recognition-and...,5.923115,0.018533,0.000408,1713.228290,389.039490,1.912226,22.613382,...,0.027460,-0.321020,4.375813,7.721463,0.081006,1.125954e-10,59.843660,0.0,9,9
15349,3,6,E:\DS5500-project\data\gesture-recognition-and...,6.466362,0.015749,0.000292,-992.810528,319.503197,2.189195,21.864578,...,0.013359,-0.493375,3.832319,1.827333,0.062739,6.332731e-11,35.905471,0.0,9,9


In [5]:
random.seed(42)
test_participants = random.sample(range(1, 44), 5)
participants = [x for x in list(range(1,44)) if x not in test_participants]

In [6]:
test_participants

[41, 8, 2, 18, 16]

In [7]:
train_df = data[data['participant'].isin(participants)]
test_df = data[data['participant'].isin(test_participants)]

In [8]:
feature_cols = [c for c in data.columns if "_" in c]

x_train = train_df.loc[:, feature_cols].values
y_train = (train_df.loc[:, 'gesture'] - 1).values
x_test = test_df.loc[:, feature_cols].values
y_test = (test_df.loc[:, 'gesture'] - 1).values

In [9]:
scaler = StandardScaler()

scaler.fit(np.vstack([x_train, x_test]))

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [10]:
lr_model = LogisticRegression(max_iter=2500, random_state=42)
lr_model.fit(x_train, y_train)

y_pred = lr_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

# Additional metrics
report = classification_report(y_test, y_pred)
print(report)

Accuracy: 78.38%
              precision    recall  f1-score   support

           0       0.63      0.59      0.61       105
           1       0.70      0.70      0.70       105
           2       0.76      0.89      0.82       105
           3       0.65      0.72      0.68       105
           4       0.74      0.70      0.72       105
           5       0.85      0.78      0.82       105
           6       0.72      0.65      0.68       105
           7       0.86      0.73      0.79       105
           8       0.69      0.64      0.66       105
           9       0.74      0.91      0.82       105
          10       0.95      0.99      0.97       105
          11       0.80      0.90      0.85       105
          12       0.76      0.87      0.81       105
          13       0.89      0.89      0.89       105
          14       0.96      0.75      0.84       105
          15       0.67      0.70      0.69       105
          16       1.00      0.92      0.96       105

    accur

In [11]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust 'n_estimators' as needed
rf_model.fit(x_train, y_train)

y_pred = rf_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

# Additional metrics
report = classification_report(y_test, y_pred)
print(report)

Accuracy: 66.50%
              precision    recall  f1-score   support

           0       0.51      0.52      0.52       105
           1       0.46      0.35      0.40       105
           2       0.49      0.51      0.50       105
           3       0.69      0.47      0.56       105
           4       0.59      0.70      0.64       105
           5       0.66      0.54      0.60       105
           6       0.57      0.62      0.59       105
           7       0.64      0.67      0.65       105
           8       0.62      0.50      0.55       105
           9       0.62      0.57      0.60       105
          10       0.81      0.99      0.89       105
          11       0.76      0.90      0.82       105
          12       0.67      0.81      0.73       105
          13       0.89      0.83      0.86       105
          14       0.75      0.83      0.79       105
          15       0.55      0.54      0.55       105
          16       0.97      0.96      0.97       105

    accur

In [12]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=2500, random_state=42)

ensemble_model = VotingClassifier(estimators=[('rf', rf_model), ('lr', lr_model)], voting='soft')
ensemble_model.fit(x_train, y_train)

y_pred = ensemble_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')


report = classification_report(y_test, y_pred)
print(report)

Accuracy: 79.55%
              precision    recall  f1-score   support

           0       0.63      0.61      0.62       105
           1       0.74      0.70      0.72       105
           2       0.76      0.89      0.82       105
           3       0.65      0.71      0.68       105
           4       0.77      0.78      0.77       105
           5       0.85      0.78      0.82       105
           6       0.72      0.70      0.71       105
           7       0.89      0.73      0.80       105
           8       0.69      0.64      0.66       105
           9       0.78      0.90      0.84       105
          10       0.95      0.99      0.97       105
          11       0.81      0.90      0.85       105
          12       0.80      0.87      0.83       105
          13       0.91      0.93      0.92       105
          14       0.98      0.76      0.86       105
          15       0.68      0.69      0.68       105
          16       1.00      0.95      0.98       105

    accur